This section contains documentation for the NCBI BLAST+ command line applications using a Docker image with BLAST+ and Jupyter Notebook server installed. We will demonstrate how to run BLAST analysis on the Google Cloud Platform using a small basic example and a more advanced production-level example. Some basic knowledge of Unix/Linux commands and BLAST+ is useful in completing this tutorial.  

In this section, We will first use the same small example from Section 1 of the official BLAST+ Docker [repository.](https://github.com/ncbi/blast_plus_docs)

Input data
* Query – 1 sequence, 44 nucleotides, file size 0.2 KB
* Database – 7 sequences, 922 nucleotides, file size 1.7 KB

### System Configurations
Region us-east4 (Northern Virginia)   
Zone us-east4c   
16vCPU 104 GB Memory n1-highmem-16   
New 200 GB standard persistent disk  
Ubuntu 18.04 LTS  

### GCP VM Cost Estimate (June 2019 - provided by GCP when VM is created)
559.96 dollars monthly ($0.767 hourly)


In [1]:
import sys

### Run BLAST+ Using a Small Example

In [3]:
!blastn -version
!efetch -version


blastn: 2.9.0+
 Package: blast 2.9.0, build Jun 10 2019 16:40:23
11.0


In [7]:
# Run BLAST+ in current directory
!mkdir -p blastdb queries fasta results blastdb_custom

In [4]:
# Retrieve sequences
!efetch -db protein -format fasta -id P01349 > queries/P01349.fsa
!efetch -db protein -format fasta -id Q90523,P80049,P83981,P83982,P83983,P83977,P83984,P83985,P27950 > fasta/nurse-shark-proteins.fsa

In [5]:
# Create BLAST databases and run BLAST
%cd blastdb_custom
!makeblastdb -in ../fasta/nurse-shark-proteins.fsa -dbtype prot -parse_seqids -out nurse-shark-proteins -title "Nurse shark proteins" -taxid 7801 -blastdb_version 5
%cd ..
!blastp -query queries/P01349.fsa -db blastdb_custom/nurse-shark-proteins

/home/mylagimail2011/jupyter_folder/blastdb_custom


Building a new DB, current time: 06/12/2019 21:33:35
New DB name:   /home/mylagimail2011/jupyter_folder/blastdb_custom/nurse-shark-proteins
New DB title:  Nurse shark proteins
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 7 sequences in 0.0144598 seconds.
/home/mylagimail2011/jupyter_folder
BLASTP 2.9.0+


Reference: Stephen F. Altschul, Thomas L. Madden, Alejandro A.
Schaffer, Jinghui Zhang, Zheng Zhang, Webb Miller, and David J.
Lipman (1997), "Gapped BLAST and PSI-BLAST: a new generation of
protein database search programs", Nucleic Acids Res. 25:3389-3402.


Reference for composition-based statistics: Alejandro A. Schaffer,
L. Aravind, Thomas L. Madden, Sergei Shavirin, John L. Spouge, Yuri
I. Wolf, Eugene V. Koonin, and Stephen F. Altschul (2001),
"Improving the accuracy of PSI-BLAST protein database searches with
composition-based statistics and other refinements", Nuclei

### Run BLAST+ at Production Scale
One of the promises of cloud computing is scalability. In this section, we will demonstrate how to use the BLAST+ Docker image at production scale on the Google Cloud Platform. We will perform a BLAST analysis similar to the approach described in this publication to compare de novo aligned contigs from bacterial 16S-23S sequencing against the nucleotide collection (nt) database.

To test scalability, we will use inputs of different sizes to estimate the amount of time to download the nucleotide collection database and run BLAST search using the latest version of the BLAST+ Docker image. Expected results are summarized in the following tables.

Input files: 28 samples (multi-FASTA files) containing de novo aligned contigs from the publication.
(Instructions to download and create the input files are described in the code block below.)

Database: Pre-formatted BLAST nucleotide collection database, version 5 (nt_v5): 68.7217 GB


* This section assumes using recommended hardware requirements - 16 CPUs, 104 GB memory and 200 GB persistent hard disk.
Modify the number of CPUs (-num_threads) if another type of VM is used.

#### Step 1. Prepare for analysis


In [ ]:
## Create directories, if not already done
!mkdir -p blastdb queries fasta results blastdb_custom


In [7]:
## Import and process input sequences

!wget https://ndownloader.figshare.com/articles/6865397?private_link=729b346eda670e9daba4 -O fa.zip
!unzip fa.zip -d fa
### Create three input query files
### All 28 samples
!cat fa/*.fa > query.fa

### Sample 1
!cat fa/'Sample_1 (paired) trimmed (paired) assembly.fa' > query1.fa

### Sample 1 to Sample 5
!cat fa/'Sample_1 (paired) trimmed (paired) assembly.fa' \
    fa/'Sample_2 (paired) trimmed (paired) assembly.fa' \
    fa/'Sample_3 (paired) trimmed (paired) assembly.fa' \
    fa/'Sample_4 (paired) trimmed (paired) assembly.fa' \
    fa/'Sample_5 (paired) trimmed (paired) assembly.fa' > query5.fa

--2019-06-12 21:36:24--  https://ndownloader.figshare.com/articles/6865397?private_link=729b346eda670e9daba4
Resolving ndownloader.figshare.com (ndownloader.figshare.com)... 18.202.190.34, 176.34.151.132, 52.30.75.146, ...
Connecting to ndownloader.figshare.com (ndownloader.figshare.com)|18.202.190.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2327955 (2.2M) [application/zip]
Saving to: ‘fa.zip’

fa.zip              100%[===================>]   2.22M   798KB/s    in 2.8s    

2019-06-12 21:36:27 (798 KB/s) - ‘fa.zip’ saved [2327955/2327955]

Archive:  fa.zip
 extracting: fa/Sample_13 (paired) trimmed (paired) assembly.fa  
 extracting: fa/Sample_14 (paired) trimmed (paired) assembly.fa  
 extracting: fa/Sample_15 (paired) trimmed (paired) assembly.fa  
 extracting: fa/Sample_16 (paired) trimmed (paired) assembly.fa  
 extracting: fa/Sample_17 (paired) trimmed (paired) assembly.fa  
 extracting: fa/Sample_18 (paired) trimmed (paired) assembly.fa  
 extract

In [9]:
### Copy query sequences to $HOME/queries folder
!cp query* queries/.

In [10]:
## Step 2. Display BLAST databases on the GCP
!update_blastdb.pl --showall pretty --source gcp


Connected to GCP
BLASTDB                                                      DESCRIPTION                                                                                                              SIZE (GB)      LAST_UPDATED
GPIPE/10090/106/GCF_000001635.24_top_level                   Mus musculus GRCm38.p4 [GCF_000001635.24] chromosomes plus unplaced and unlocalized scaffolds                               1.4707      2016-06-22
GPIPE/9606/109/GCF_000001405.38_top_level                    Homo sapiens GRCh38.p12 [GCF_000001405.38] chromosomes plus unplaced and unlocalized scaffolds                              1.7225      2018-03-28
SMARTBLAST/landmark_v5                                       Landmark database for SmartBLAST                                                                                            0.5262      2019-03-19
genomic/Viruses/NCBI_VIV_nucleotide_sequences_v5             NCBI VIV nucleotide sequences                                                           

In [12]:
## Download nt_v5 (nucleotide collection version 5) database
## This step takes approximately 9 min.  Background processes not supported
%cd blastdb
!update_blastdb.pl --source gcp nt_v5 
%cd ..

[Errno 2] No such file or directory: 'blastdb'
/home/mylagimail2011/jupyter_folder/blastdb
Connected to GCP
/home/mylagimail2011/jupyter_folder


At this point, confirm query/database have been properly provisioned before proceeding.  
First, check the size of the directory containing the BLAST database
(nt_v5 should be around 68 GB) In addition, there should be three files in the `queries` folder - `query.fa`, `query1.fa`, and `query5.fa`.

In [13]:
!du -sk blastdb
!ls -al queries

69162420	blastdb
total 2756
drwxr-xr-x 2 jovyan users    4096 Jun 12 21:37 .
drwxrwxrwx 9   1001  1002    4096 Jun 12 21:46 ..
-rw-r--r-- 1 jovyan users     173 Jun 12 21:33 P01349.fsa
-rw-r--r-- 1 jovyan users   58804 Jun 12 21:37 query1.fa
-rw-r--r-- 1 jovyan users  422279 Jun 12 21:37 query5.fa
-rw-r--r-- 1 jovyan users 2321683 Jun 12 21:37 query.fa


#### Step 3. Run BLAST

First, run BLAST using `query1.fa`. (Sample 1) 
This command will take approximately 8 minutes to complete and create a 3.1 GB output file. 

In [14]:
!blastn -query queries/query1.fa -db blastdb/nt_v5 -num_threads 16 -out results/blastn.query1.denovo16s.out


Segmentation fault (core dumped)


Next, run BLAST using query5.fa (Samples 1-5) 
This command will take approximately 27 minutes to complete and create a 10.4 GB output file. 

In [ ]:
!blastn -query queries/query5.fa -db blastdb/nt_v5 -num_threads 16 -out results/blastn.query5.denovo16s.out



Finally, run BLAST using `query.fa` (All samples) This command will take approximately 140 minutes to complete and create a 47.8 GB output file.


In [ ]:
!blastn -query queries/query.fa -db blastdb/nt_v5 -num_threads 16 -out results/blastn.query.denovo16s.out


### Stop the GCP instance
Remember to stop or delete the VM to prevent incurring additional cost. You can do this at the GCP Console.

